In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
file_path = '/content/BenchmarkUddinSO-ConsoliatedAspectSentiment.xls'
data = pd.read_excel(file_path)

# Selecting and preparing the relevant columns
data = data[['codes', 'sent', 'ManualLabel']]
data['codes'] = data['codes'].apply(eval)
data = data.explode('codes')
data = data[data['codes'].str.contains('Bug')]  # Filter to only 'Bug' aspect

# Mapping labels to new non-negative numerical values
label_mapping = {'p': 2, 'n': 1, 'o': 0}  # Positive, Negative, Other
data['ManualLabel'] = data['ManualLabel'].map(label_mapping)

In [ ]:
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(data['sent'], data['ManualLabel'], test_size=0.3, random_state=42)

# Tokenizer and model setup
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)  # for three labels p, n, o

# Dataset class
class ReviewDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# DataLoader setup
def create_data_loader(X, y, tokenizer, batch_size, max_len):
    ds = ReviewDataset(
        texts=X.to_numpy(),
        labels=y.to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
    )
    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=0  # Set workers to 0 to avoid multiprocessing issues in certain environments
    )

batch_size = 16
train_data_loader = create_data_loader(X_train, y_train, tokenizer, batch_size, max_len=128)
test_data_loader = create_data_loader(X_test, y_test, tokenizer, batch_size, max_len=128)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training Loop
def train_epoch(model, data_loader, optimizer, device, n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0

    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss
        preds = torch.argmax(outputs.logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    return correct_predictions.double() / n_examples, sum(losses) / len(losses)

# Train the model
for epoch in range(3):  # Number of epochs
    print(f'Epoch {epoch + 1}')
    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,
        optimizer,
        device,
        len(X_train)
    )
    print(f'Train loss {train_loss} accuracy {train_acc}')

Epoch 1
Train loss 1.098270833492279 accuracy 0.45454545454545453
Epoch 2
Train loss 0.9275918934080336 accuracy 0.5909090909090909
Epoch 3
Train loss 0.8297905921936035 accuracy 0.7045454545454546


In [ ]:
import time
from sklearn.metrics import accuracy_score, f1_score

# Set the model to evaluation mode
model.eval()
predictions, true_labels = [], []
total_inference_time = 0
total_samples = 0

# Evaluation loop
with torch.no_grad():
    for batch in test_data_loader:  # Make sure to use the correct DataLoader variable name
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        start_time = time.time()
        outputs = model(input_ids, attention_mask=attention_mask)
        inference_time = time.time() - start_time
        total_inference_time += inference_time
        total_samples += input_ids.size(0)

        logits = outputs.logits
        preds = logits.argmax(dim=1).cpu().numpy()
        labels = batch['labels'].cpu().numpy()

        predictions.extend(preds)
        true_labels.extend(labels)

# Compute metrics
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions, average='weighted')  # Ensure to specify the average method if applicable

print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')

# Calculate average inference time per sample
average_inference_time = total_inference_time / total_samples
print(f'Total Inference Time: {total_inference_time:.6f} seconds')
print(f'Total Samples: {total_samples}')
print(f'Average Inference Time per Sample: {average_inference_time:.6f} seconds')


Accuracy: 0.5614
F1 Score: 0.5111
Total Inference Time: 31.041202 seconds
Total Samples: 57
Average Inference Time per Sample: 0.544582 seconds
